In [17]:
import numpy as np
import cv2
import sys

inputSize = [513, 513];
mean = [127.5, 127.5, 127.5];
std = 0.007843;
rb = False
cls = 0
path = './opt_deeplabv3_mnv2_513.pb'

#std = 1/255.0
#inputSize = (1024,512)
#mean = 0
#rb = True
#cls = 13
#path = './model-cityscapes.net'

sample_img = cv2.imread('./fig/face1.jpg')
back = cv2.imread("./fig/wall.jpg")
r_back = cv2.resize(back,inputSize,interpolation=cv2.INTER_LINEAR)
deeplab_model = cv2.dnn.readNet(path)

cap = cv2.VideoCapture(0)

if not cap.isOpened() :
    print('camera open fail')
    sys.exit()

    

fps = int(cap.get(cv2.CAP_PROP_FPS))
focc = cv2.VideoWriter_fourcc(*"DIVX")
#fourcc video codec style default DIVX
print("width : {}, height : {} fps:{}".format(inputSize[0],inputSize[1],fps))

out = cv2.VideoWriter('segmentation_outcom.avi',focc,fps,inputSize,True)    
    
while True:
    rat,frame = cap.read()
    if not rat:
        print("frame read fail")
        sys.exit()
        
    copy_frame = frame.copy()
    blob_img = cv2.dnn.blobFromImage(copy_frame, std, inputSize, mean, swapRB=rb, crop=False )
    deeplab_model.setInput(blob_img)
    deeplab_model_output = deeplab_model.forward()
    class_map = np.argmax(deeplab_model_output[0],axis=0)
    class_map = np.where(class_map == cls,1,0)
    IMG_COLORS = np.array([0,255])
    mask = IMG_COLORS[class_map].astype(np.uint8)
    resize_frame = cv2.resize(copy_frame,inputSize,interpolation=cv2.INTER_LINEAR)
    resize_back = r_back.copy()
    cv2.copyTo(resize_frame,mask,resize_back)
    
    cv2.imshow("window",resize_back)
    out.write(resize_back)
    key = cv2.waitKey(100)
    if key == 27:
        break

out.release()
cap.release()
cv2.destroyAllWindows()


width : 513, height : 513 fps:30
